In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.collections import LineCollection

from utils import get_rect_to_rect_data, get_rect_coords_from_center_point

In [ ]:
ped_files = ['VM2_-2112701535', 'VM2_-217686115','VM2_-1247665811','VM2_-104300786', 'VM2_330973206', 'VM2_1100569031', 'VM2_421371629', 'VM2_421371629', 'VM2_1476499235', 'VM2_-1451152685', 'VM2_-1523872256']

In [ ]:
# from matplotlib import rc

# rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 16})
# rc('text', usetex=True)

In [ ]:

start_point_coords = (52.52181, 13.41645)
end_point_coords = (52.52231, 13.41703)

start_rect_coords = get_rect_coords_from_center_point(start_point_coords)
end_rect_coords = get_rect_coords_from_center_point(end_point_coords)

# start_rect_coords = (13.416354, 52.521752, 13.416705, 52.521961)
# end_rect_coords = (13.416912, 52.522261, 13.417263, 52.52247)

end_date_str = '2021-01-01 00:00:00'
end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')

df_simra = get_rect_to_rect_data(start_rect_coords, end_rect_coords, end_date=end_date)
df_simra


In [ ]:
df_simra['lon'], df_simra['lat'] = zip(*df_simra.coords.values)

fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(df_simra.velos.min(), df_simra.velos.max())

df_simra_grouped = df_simra.groupby('filename')
for ride_group_name in df_simra_grouped.groups:
    df_ride_group = df_simra_grouped.get_group(ride_group_name)
    points = np.array([df_ride_group.lon, df_ride_group.lat]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments, cmap='viridis', norm=norm)
    # Set the values used for colormapping
    lc.set_array(df_ride_group.velos.values)
    lc.set_linewidth(2)
    line = ax.add_collection(lc)

ax.set_xlim(min(df_simra.lon), max(df_simra.lon))
ax.set_ylim(min(df_simra.lat), max(df_simra.lat))

axcb = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.1)
axcb.set_label('Velocity in m/s')

ax.xaxis.set_major_locator(ticker.LinearLocator(4))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.yaxis.set_major_locator(ticker.LinearLocator(4))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax.set_xlabel('Longitude in decimal degrees')
ax.set_ylabel('Latitude in decimal degrees')


plt.show()

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import io
from urllib.request import urlopen, Request
from PIL import Image

def plot_location(x, color, alpha, scatter, thickness):
    lon = []
    lat = []
    for c in x.coords.values:
        lon.append(c[0])
        lat.append(c[1])
    plt.plot(lon, lat, c=color, alpha=alpha, linewidth=thickness)
    print(thickness)
    if scatter:
        zero_v_coords = x.apply(lambda x: x.coords if x.velos == 0 else None,axis = 1)
        lon = []
        lat = []
        for c in zero_v_coords.values:
            if c == None:
                continue
            lon.append(c[0])
            lat.append(c[1])
        plt.scatter(lon, lat)

def image_spoof(self, tile): # this function pretends not to be a Python script
    url = self._image_url(tile) # get the url of the street map API
    req = Request(url) # start request
    req.add_header('User-agent','Anaconda 3') # add user agent to request
    fh = urlopen(req) 
    im_data = io.BytesIO(fh.read()) # get image
    fh.close() # close url
    img = Image.open(im_data) # open image with PIL
    img = img.convert(self.desired_tile_form) # set image format
    return img, self.tileextent(tile), 'lower' # reformat for cartopy

cimgt.OSM.get_image = image_spoof # reformat web request for street map spoofing
osm_img = cimgt.OSM() # spoofed, downloaded street map

fig = plt.figure(figsize=(24,18)) # open matplotlib figure
ax1 = plt.axes(projection=osm_img.crs) # project using coordinate reference system (CRS) of street map
#center_pt = [52.520008, 13.404954] # lat/lon of One World Trade Center in NYC
center_pt = [52.522203332461444, 13.416717768136085]
zoom = 0.00075 # for zooming out of center point
extent = [center_pt[1]-(zoom*2.0),center_pt[1]+(zoom*2.0),center_pt[0]-zoom,center_pt[0]+zoom] # adjust to zoom
ax1.set_extent(extent) # set extents

scale = np.ceil(-np.sqrt(2)*np.log(np.divide(zoom,350.0))) # empirical solve for scale based on zoom
scale = (scale<20) and scale or 19 # scale cannot be larger than 19
ax1.add_image(osm_img, int(scale)) # add OSM with zoom specification

colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c']
df_simra.groupby('filename').apply(lambda x: plot_location(x, colors[0], 0.5, False, 10000000))
#print(df_simra)

# NOTE: zoom specifications should be selected based on extent:
# -- 2     = coarse image, select for worldwide or continental scales
# -- 4-6   = medium coarseness, select for countries and larger states
# -- 6-10  = medium fineness, select for smaller states, regions, and cities
# -- 10-12 = fine image, select for city boundaries and zip codes
# -- 14+   = extremely fine image, select for roads, blocks, buildings
plt.show() # show the plot